In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
import os
import json
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

Data splitting:
- Split the entire dataset into training and testing sets first.
- Use the training set to generate folds (one validation and the rest training folds) (K-Fold Cross Validation)

In [8]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=REPRODUCIBILITY_SEED)
print(f"Training set size: {len(train_data)} | Test set size: {len(test_data)}")    
print()


Training set size: 8000 | Test set size: 2000



In [9]:
normaliser = Normaliser()
os.makedirs(TRAINING_STATISTICS_DIR, exist_ok=True)
stats_for_each_column = {}
for column in numeric_columns:
    print(data[column])
    train_data_column_mean = normaliser.calculate_mean(train_data[column])
    train_data_column_std = normaliser.calculate_std(train_data[column])

    train_data[column] = normaliser.standardise(train_data[column], mean=train_data_column_mean, std=train_data_column_std)

    stats_for_each_column[column] = {
        "mean": train_data_column_mean,
        "std": train_data_column_std
    }

    # Normalise test data using the mean and std of the training data
    test_data[column] = normaliser.standardise(test_data[column], mean=train_data_column_mean, std=train_data_column_std)
    print("after", train_data[column])

with open(f"{TRAINING_STATISTICS_DIR}/stats.json", "w") as f:
    json.dump(stats_for_each_column, f)

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 9254   -1.023878
1561    1.522047
1670   -1.045094
6087   -0.111588
6669    1.140158
          ...   
5734   -0.154021
5191    0.970430
5390   -1.045094
860    -1.002662
7270   -1.151175
Name: carat, Length: 8000, dtype: float64
0       62.3
1       60.5
2       60.7
3       61.2
4       62.4
        ... 
9995    62.6
9996    69.5
9997    57.9
9998    62.0
9999    59.9
Name: depth, Length: 10000, dtype: float64
after 9254   -0.381342
1561    0.945157
1670    0.386631
6087   -0.311527
6669   -0.311527
          ...   
5734    0.665894
5191    0.107368
5390    0.107368
860     0.875341
7270    0.665894
Name: depth, Length: 8000, dtype: float64
0       56.0
1       59.0
2       58.0
3       58.0
4       54.0
        ... 
9995    57.0
9996    55.0
9997    61.0
9998    54.0
9999    59.0
Name: table, Le

In [10]:
kfold_data = get_kfold_data(data=train_data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5',
       'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10',
       'colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J'],
      dtype='object')
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5',
       'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10',
       'colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J'],
      dtype='object')

Fold: 1/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%
Index(['outcome', 'carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', '

Define models and hyperparameter tuning objectives for each model

In [11]:
models = {
        "linear_regression": LinearRegression,
        "lasso": Lasso,
        "ridge": Ridge,
        "xgb": xgb.XGBRegressor,
        "random_forest": RandomForestRegressor,
        "gradient_boosting": GradientBoostingRegressor,
        "ada_boost": AdaBoostRegressor,
        "lgbm": lgb.LGBMRegressor
        }

# Must contain the one-hot encoded columns (otherwise this does not make sense)
minimum_features = [feature for feature in data.columns if feature.startswith("colour")] 

def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == LinearRegression:
        parameters = {
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        }
    elif model_type == Lasso:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "selection": trial.suggest_categorical("selection", ["cyclic", "random"]),
            "warm_start": trial.suggest_categorical("warm_start", [True, False]),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Ridge:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "solver": trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "positive": False,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == GradientBoostingRegressor:
        parameters = {
            "n_estimators": 100,
            "loss": trial.suggest_categorical("loss", ["absolute_error", "squared_error", "huber", "quantile"]),
            "criterion": trial.suggest_categorical("criterion", ["friedman_mse", "squared_error"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 2**10),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == AdaBoostRegressor:
        parameters = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }
    
    # Wrapper method for feature selection
    selected_features = minimum_features.copy()
    train_columns = x_train.columns.tolist() 
    for feature in train_columns:
        if feature.startswith("colour"):
            continue
        if trial.suggest_categorical(feature, [0, 1]) == 1:
            selected_features.append(feature)
    if len(selected_features) == 0:
        return float("inf")
    
    x_train = x_train[selected_features]
    x_val = x_val[selected_features]

    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

if os.path.exists("model_best_hyperparameters"):
    raise Exception("Directory for best hyperparameters already exists. Please delete it before running this script.")

os.makedirs("model_best_hyperparameters")

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    
    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")
    training_features = set(train_x.columns.tolist())


    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        print("G", train_x.columns)
        print("G", val_x.columns)
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=N_TRIALS)
        
        # Train model with best hyperparameters
        best_trial = study.best_trial 

        best_fold_params = {} # Hyperparameters
        best_selected_features = minimum_features.copy() # Features selected by the model
        for param in best_trial.params:
            if param in training_features: # i.e., if the parameter is a feature
                if best_trial.params[param] == 1:
                    best_selected_features.append(param)
            else: # A hyperparameter
                best_fold_params[param] = best_trial.params[param]
        print("P", best_fold_params)
        print("F", best_selected_features)
        
        # Select the best features
        print(train_x.columns)
        print(val_x.columns)
        train_x = train_x[best_selected_features]
        val_x = val_x[best_selected_features]

        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)
        
        # Save the best hyperparameters for this model at this fold.
        os.makedirs(f"model_best_hyperparameters/{model_name}", exist_ok=True)
        with open(f"model_best_hyperparameters/{model_name}/fold_{fold+1}.json", "w") as f:
            json.dump(best_fold_params, f)
        with open(f"model_best_hyperparameters/{model_name}/fold_{fold+1}_selected_features.json", "w") as f:
            json.dump(best_selected_features, f)

        # Calculate metrics
        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()


[I 2025-02-18 16:22:22,899] A new study created in memory with name: no-name-57bca8b4-71e4-4bb1-a47a-93ee1f6efeab
[I 2025-02-18 16:22:22,907] Trial 0 finished with value: 12.079737445821111 and parameters: {'fit_intercept': False, 'carat': 1, 'cut': 1, 'clarity': 1, 'depth': 0, 'table': 1, 'price': 0, 'x': 0, 'y': 0, 'z': 1, 'a1': 1, 'a2': 1, 'a3': 0, 'a4': 1, 'a5': 0, 'b1': 1, 'b2': 0, 'b3': 0, 'b4': 1, 'b5': 1, 'a6': 0, 'a7': 1, 'a8': 1, 'a9': 1, 'a10': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'b9': 0, 'b10': 0}. Best is trial 0 with value: 12.079737445821111.
[I 2025-02-18 16:22:22,913] Trial 1 finished with value: 12.065051961136902 and parameters: {'fit_intercept': True, 'carat': 1, 'cut': 0, 'clarity': 0, 'depth': 0, 'table': 1, 'price': 1, 'x': 0, 'y': 1, 'z': 0, 'a1': 1, 'a2': 0, 'a3': 0, 'a4': 1, 'a5': 0, 'b1': 0, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 0, 'a6': 0, 'a7': 0, 'a8': 1, 'a9': 1, 'a10': 0, 'b6': 1, 'b7': 0, 'b8': 1, 'b9': 1, 'b10': 1}. Best is trial 1 with value: 12.065051961136902.

G Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
G Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')


[I 2025-02-18 16:22:22,980] Trial 10 finished with value: 11.274102604911025 and parameters: {'fit_intercept': False, 'carat': 1, 'cut': 0, 'clarity': 1, 'depth': 1, 'table': 0, 'price': 1, 'x': 0, 'y': 0, 'z': 0, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 0, 'a5': 1, 'b1': 0, 'b2': 1, 'b3': 1, 'b4': 0, 'b5': 0, 'a6': 1, 'a7': 1, 'a8': 0, 'a9': 1, 'a10': 1, 'b6': 0, 'b7': 0, 'b8': 0, 'b9': 1, 'b10': 0}. Best is trial 6 with value: 10.972781715240504.
[I 2025-02-18 16:22:22,991] Trial 11 finished with value: 11.161148621984024 and parameters: {'fit_intercept': False, 'carat': 1, 'cut': 0, 'clarity': 0, 'depth': 1, 'table': 0, 'price': 0, 'x': 1, 'y': 0, 'z': 0, 'a1': 0, 'a2': 0, 'a3': 1, 'a4': 1, 'a5': 1, 'b1': 0, 'b2': 0, 'b3': 1, 'b4': 0, 'b5': 1, 'a6': 1, 'a7': 0, 'a8': 0, 'a9': 0, 'a10': 0, 'b6': 0, 'b7': 0, 'b8': 1, 'b9': 0, 'b10': 1}. Best is trial 6 with value: 10.972781715240504.
[I 2025-02-18 16:22:23,003] Trial 12 finished with value: 11.155397495245769 and parameters: {'fit_intercept':

P {'fit_intercept': False}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'carat', 'cut', 'clarity', 'depth', 'table', 'x', 'z', 'a1', 'a4', 'b1', 'b3', 'b5', 'a6', 'a8', 'a9', 'b6', 'b8']
Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
Index(['carat', 'cut', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z',
       'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7',
       'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10', 'colour_G',
       'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J'],
      dtype='object')
Fold: 1/5
Model name: linear_regression
MAE: 8.678654058165534
MSE: 115.8232988977597
RMSE: 10.762123345221411
PCC: 0.5

[I 2025-02-18 16:22:24,397] Trial 17 finished with value: 10.949030495371465 and parameters: {'alpha': 0.007117805241283343, 'fit_intercept': True, 'selection': 'cyclic', 'warm_start': False, 'carat': 1, 'cut': 1, 'clarity': 0, 'depth': 1, 'table': 0, 'x': 0, 'z': 1, 'a1': 1, 'a4': 1, 'b1': 0, 'b3': 1, 'b5': 0, 'a6': 0, 'a8': 1, 'a9': 0, 'b6': 1, 'b8': 1}. Best is trial 15 with value: 10.944411388759402.
[I 2025-02-18 16:22:24,407] Trial 18 finished with value: 11.325886818753085 and parameters: {'alpha': 0.09683948040165082, 'fit_intercept': True, 'selection': 'cyclic', 'warm_start': False, 'carat': 0, 'cut': 1, 'clarity': 0, 'depth': 1, 'table': 0, 'x': 0, 'z': 1, 'a1': 0, 'a4': 1, 'b1': 1, 'b3': 0, 'b5': 0, 'a6': 0, 'a8': 0, 'a9': 0, 'b6': 1, 'b8': 0}. Best is trial 15 with value: 10.944411388759402.
[I 2025-02-18 16:22:24,418] Trial 19 finished with value: 10.949660090059766 and parameters: {'alpha': 0.0027021772939990387, 'fit_intercept': True, 'selection': 'cyclic', 'warm_start':

...

P {'n_estimators': 91, 'loss': 'linear', 'learning_rate': 0.08315202481660834}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'depth', 'a1', 'b1', 'b3']
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')


[I 2025-02-18 16:34:03,888] A new study created in memory with name: no-name-a0a30309-7554-4d58-b3a6-102aaa1091e0
[I 2025-02-18 16:34:03,980] Trial 0 finished with value: 12.365761235616683 and parameters: {'learning_rate': 0.0013056346865094244, 'num_leaves': 911, 'subsample': 0.5696596270686342, 'colsample_bytree': 0.6411080986155429, 'min_data_in_leaf': 54, 'depth': 0, 'a1': 0, 'b1': 1, 'b3': 1}. Best is trial 0 with value: 12.365761235616683.
[I 2025-02-18 16:34:04,024] Trial 1 finished with value: 11.92547960142028 and parameters: {'learning_rate': 0.09205142532450777, 'num_leaves': 496, 'subsample': 0.5942540362423929, 'colsample_bytree': 0.4923198811487761, 'min_data_in_leaf': 89, 'depth': 0, 'a1': 0, 'b1': 1, 'b3': 1}. Best is trial 1 with value: 11.92547960142028.


Fold: 5/5
Model name: ada_boost
MAE: 7.440195009243428
MSE: 87.46335204701604
RMSE: 9.3521843462913
PCC: 0.6615139955044345
Spearman R: 0.6690527578664317
R2 Score: 0.43689699728627507

G Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')
G Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')


[I 2025-02-18 16:34:04,068] Trial 2 finished with value: 11.963470657145555 and parameters: {'learning_rate': 0.0014343689306455273, 'num_leaves': 362, 'subsample': 0.651834609464172, 'colsample_bytree': 0.9293237462147566, 'min_data_in_leaf': 89, 'depth': 1, 'a1': 0, 'b1': 0, 'b3': 0}. Best is trial 1 with value: 11.92547960142028.
[I 2025-02-18 16:34:04,100] Trial 3 finished with value: 12.047460355123652 and parameters: {'learning_rate': 0.0012103755238531172, 'num_leaves': 780, 'subsample': 0.21772630591150227, 'colsample_bytree': 0.7022299665512746, 'min_data_in_leaf': 88, 'depth': 1, 'a1': 1, 'b1': 0, 'b3': 1}. Best is trial 1 with value: 11.92547960142028.
[I 2025-02-18 16:34:04,278] Trial 4 finished with value: 11.768806141763777 and parameters: {'learning_rate': 0.0022348691171688974, 'num_leaves': 463, 'subsample': 0.36549231555392475, 'colsample_bytree': 0.7939728461902109, 'min_data_in_leaf': 6, 'depth': 1, 'a1': 0, 'b1': 0, 'b3': 0}. Best is trial 4 with value: 11.76880614

P {'learning_rate': 0.04152185578142371, 'num_leaves': 1017, 'subsample': 0.5128426639228317, 'colsample_bytree': 0.9039894149692504, 'min_data_in_leaf': 83}
F ['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I', 'colour_J', 'depth', 'a1', 'b1', 'b3']
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')
Index(['colour_G', 'colour_E', 'colour_F', 'colour_H', 'colour_D', 'colour_I',
       'colour_J', 'depth', 'a1', 'b1', 'b3'],
      dtype='object')
Fold: 5/5
Model name: lgbm
MAE: 7.367489131844955
MSE: 84.87467280576088
RMSE: 9.212745128666096
PCC: 0.6738416221302715
Spearman R: 0.6778058126194582
R2 Score: 0.45356332689401624



Compute average scores and rank models by R2 score

In [13]:
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])

model_scores = dict(sorted(model_scores.items(), key=lambda x: x[1]["r2_score"], reverse=True))

In [14]:
# Print results
for i, (model_name, model_metrics) in enumerate(model_scores.items()):
    print(f"No.{i+1} Model: {model_name}")
    for metric, score in model_metrics.items():
        print(f"{metric}: {score}")
    print()

No.1 Model: xgb
mae: 7.4665103628458285
mse: 88.26313397339472
rmse: 9.392898849030006
pcc: 0.6736511773184859
spearman_r: 0.6788629212449057
r2_score: 0.453343025967731

No.2 Model: gradient_boosting
mae: 7.535193065755017
mse: 90.09082681895033
rmse: 9.489693900396853
pcc: 0.6650296256442079
spearman_r: 0.6700441249837938
r2_score: 0.4420635649845418

No.3 Model: random_forest
mae: 7.574612361701929
mse: 90.7530398251658
rmse: 9.52486721504172
pcc: 0.6638123418782502
spearman_r: 0.6685024369150143
r2_score: 0.43799839899956633

No.4 Model: lgbm
mae: 7.659961928638319
mse: 92.84562849144984
rmse: 9.63096345761997
pcc: 0.653037309046761
spearman_r: 0.6560131177657038
r2_score: 0.42508268631753526

No.5 Model: ada_boost
mae: 7.671529511514753
mse: 93.14583480038098
rmse: 9.649717860044161
pcc: 0.6512398170101837
spearman_r: 0.658004641335425
r2_score: 0.4231882171845879

No.6 Model: linear_regression
mae: 8.550944294022955
mse: 114.58154754122492
rmse: 10.701961581717097
pcc: 0.53969696